# Modeling

In [2]:
# load libraries

import numpy as np
import pandas as pd
import geopandas as gpd

from matplotlib import pyplot as plt
%matplotlib inline
import altair as alt

from sklearn.cluster import KMeans, dbscan 
from sklearn.preprocessing import StandardScaler


In [3]:
plt.rcParams['figure.figsize'] = (10,6)

alt.renderers.enable("notebook")
np.random.seed(42)

In [4]:
path_data = './data/Y2019M07D12_Aqueduct30_V01/baseline/annual/y2019m07d11_aqueduct30_annual_v01.gpkg'

data = gpd.read_file(path_data, layer='y2019m07d11_aqueduct30_annual_v01')

data

,string_id,aq30_id,pfaf_id,gid_1,aqid,gid_0,name_0,name_1,area_km2,bws_raw,...,w_awr_tex_rrr_score,w_awr_tex_rrr_cat,w_awr_tex_rrr_label,w_awr_tex_rrr_weight_fraction,w_awr_tex_tot_raw,w_awr_tex_tot_score,w_awr_tex_tot_cat,w_awr_tex_tot_label,w_awr_tex_tot_weight_fraction,geometry
0,111011-EGY.11_1-3365,0,111011,EGY.11_1,3365,EGY,Egypt,Al Qahirah,4.223754,2.318042,...,2.165272,2.0,Medium - High (2-3),0.326531,2.457171,3.614603,3.0,High (3-4),0.877551,(POLYGON ((31.90590570688292 29.85788703615783...
1,111011-EGY.15_1-3365,1,111011,EGY.15_1,3365,EGY,Egypt,As Suways,1846.012343,2.318042,...,2.165272,2.0,Medium - High (2-3),0.326531,2.457171,3.614603,3.0,High (3-4),0.877551,(POLYGON ((31.90256665452753 29.82957728496378...
2,111011-EGY.15_1-None,2,111011,EGY.15_1,-9999,EGY,Egypt,As Suways,30.526067,2.318042,...,2.165272,2.0,Medium - High (2-3),0.326531,2.457171,3.614603,3.0,High (3-4),0.877551,(POLYGON ((32.36609158210641 29.63378296420298...
3,111011-None-3365,3,111011,-9999,3365,None,None,None,0.742712,2.318042,...,1.133763,1.0,Low - Medium (1-2),0.163265,2.477321,3.649648,3.0,High (3-4),0.551020,(POLYGON ((32.46194054146073 29.89250514754305...
4,111011-None-None,4,111011,-9999,-9999,None,None,None,13.430995,2.318042,...,1.133763,1.0,Low - Medium (1-2),0.163265,2.477321,3.649648,3.0,High (3-4),0.551020,(POLYGON ((32.46439791820441 29.88855938783929...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68501,None-YEM.5_1-None,68506,-9999,YEM.5_1,-9999,YEM,Yemen,Al Hudaydah,0.887022,NaN,...,4.626502,4.0,Extremely High (4-5),0.163265,4.277528,4.688755,4.0,Extremely High (4-5),0.326531,(POLYGON ((42.92231750015262 14.94162846008543...
68502,None-ZAF.1_1-None,68507,-9999,ZAF.1_1,-9999,ZAF,South Africa,Eastern Cape,0.019523,NaN,...,2.177403,2.0,Medium - High (2-3),0.163265,1.822667,2.430366,2.0,Medium - High (2-3),0.326531,(POLYGON ((25.13501089713526 -33.9629173298708...
68503,None-ZAF.4_1-None,68508,-9999,ZAF.4_1,-9999,ZAF,South Africa,KwaZulu-Natal,0.085616,NaN,...,2.177403,2.0,Medium - High (2-3),0.163265,1.822667,2.430366,2.0,Medium - High (2-3),0.326531,(POLYGON ((32.09148190682467 -28.8150333751501...
68504,None-ZAF.9_1-2940,68509,-9999,ZAF.9_1,2940,ZAF,South Africa,Western Cape,0.015411,NaN,...,2.177403,2.0,Medium - High (2-3),0.163265,1.822667,2.430366,2.0,Medium - High (2-3),0.326531,(POLYGON ((22.57251155246618 -33.9962501502242...


In [7]:
pd.set_option('display.max_columns', None)
data.head(1)

,string_id,aq30_id,pfaf_id,gid_1,aqid,gid_0,name_0,name_1,area_km2,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,gtd_raw,gtd_score,gtd_cat,gtd_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_score,rri_cat,rri_label,w_awr_def_qan_raw,w_awr_def_qan_score,w_awr_def_qan_cat,w_awr_def_qan_label,w_awr_def_qan_weight_fraction,w_awr_def_qal_raw,w_awr_def_qal_score,w_awr_def_qal_cat,w_awr_def_qal_label,w_awr_def_qal_weight_fraction,w_awr_def_rrr_raw,w_awr_def_rrr_score,w_awr_def_rrr_cat,w_awr_def_rrr_label,w_awr_def_rrr_weight_fraction,w_awr_def_tot_raw,w_awr_def_tot_score,w_awr_def_tot_cat,w_awr_def_tot_label,w_awr_def_tot_weight_fraction,w_awr_agr_qan_raw,w_awr_agr_qan_score,w_awr_agr_qan_cat,w_awr_agr_qan_label,w_awr_agr_qan_weight_fraction,w_awr_agr_qal_raw,w_awr_agr_qal_score,w_awr_agr_qal_cat,w_awr_agr_qal_label,w_awr_agr_qal_weight_fraction,w_awr_agr_rrr_raw,w_awr_agr_rrr_score,w_awr_agr_rrr_cat,w_awr_agr_rrr_label,w_awr_agr_rrr_weight_fraction,w_awr_agr_tot_raw,w_awr_agr_tot_score,w_awr_agr_tot_cat,w_awr_agr_tot_label,w_awr_agr_tot_weight_fraction,w_awr_che_qan_raw,w_awr_che_qan_score,w_awr_che_qan_cat,w_awr_che_qan_label,w_awr_che_qan_weight_fraction,w_awr_che_qal_raw,w_awr_che_qal_score,w_awr_che_qal_cat,w_awr_che_qal_label,w_awr_che_qal_weight_fraction,w_awr_che_rrr_raw,w_awr_che_rrr_score,w_awr_che_rrr_cat,w_awr_che_rrr_label,w_awr_che_rrr_weight_fraction,w_awr_che_tot_raw,w_awr_che_tot_score,w_awr_che_tot_cat,w_awr_che_tot_label,w_awr_che_tot_weight_fraction,w_awr_con_qan_raw,w_awr_con_qan_score,w_awr_con_qan_cat,w_awr_con_qan_label,w_awr_con_qan_weight_fraction,w_awr_con_qal_raw,w_awr_con_qal_score,w_awr_con_qal_cat,w_awr_con_qal_label,w_awr_con_qal_weight_fraction,w_awr_con_rrr_raw,w_awr_con_rrr_score,w_awr_con_rrr_cat,w_awr_con_rrr_label,w_awr_con_rrr_weight_fraction,w_awr_con_tot_raw,w_awr_con_tot_score,w_awr_con_tot_cat,w_awr_con_tot_label,w_awr_con_tot_weight_fraction,w_awr_elp_qan_raw,w_awr_elp_qan_score,w_awr_elp_qan_cat,w_awr_elp_qan_label,w_awr_elp_qan_weight_fraction,w_awr_elp_qal_raw,w_awr_elp_qal_score,w_awr_elp_qal_cat,w_awr_elp_qal_label,w_awr_elp_qal_weight_fraction,w_awr_elp_rrr_raw,w_awr_elp_rrr_score,w_awr_elp_rrr_cat,w_awr_elp_rrr_label,w_awr_elp_rrr_weight_fraction,w_awr_elp_tot_raw,w_awr_elp_tot_score,w_awr_elp_tot_cat,w_awr_elp_tot_label,w_awr_elp_tot_weight_fraction,w_awr_fnb_qan_raw,w_awr_fnb_qan_score,w_awr_fnb_qan_cat,w_awr_fnb_qan_label,w_awr_fnb_qan_weight_fraction,w_awr_fnb_qal_raw,w_awr_fnb_qal_score,w_awr_fnb_qal_cat,w_awr_fnb_qal_label,w_awr_fnb_qal_weight_fraction,w_awr_fnb_rrr_raw,w_awr_fnb_rrr_score,w_awr_fnb_rrr_cat,w_awr_fnb_rrr_label,w_awr_fnb_rrr_weight_fraction,w_awr_fnb_tot_raw,w_awr_fnb_tot_score,w_awr_fnb_tot_cat,w_awr_fnb_tot_label,w_awr_fnb_tot_weight_fraction,w_awr_min_qan_raw,w_awr_min_qan_score,w_awr_min_qan_cat,w_awr_min_qan_label,w_awr_min_qan_weight_fraction,w_awr_min_qal_raw,w_awr_min_qal_score,w_awr_min_qal_cat,w_awr_min_qal_label,w_awr_min_qal_weight_fraction,w_awr_min_rrr_raw,w_awr_min_rrr_score,w_awr_min_rrr_cat,w_awr_min_rrr_label,w_awr_min_rrr_weight_fraction,w_awr_min_tot_raw,w_awr_min_tot_score,w_awr_min_tot_cat,w_awr_min_tot_label,w_awr_min_tot_weight_fraction,w_awr_ong_qan_raw,w_awr_ong_qan_score,w_awr_ong_qan_cat,w_awr_ong_qan_label,w_awr_ong_qan_weight_fraction,w_awr_ong_qal_raw,w_awr_ong_qal_score,w_awr_ong_qal_cat,w_awr_ong_qal_label,w_awr_ong_qal_weight_fraction,w_awr_ong_rrr_raw,w_awr_ong_rrr_score,w_awr_ong_rrr_cat,w_awr_ong_rrr_label,w_awr_ong_rrr_weight_fraction,w_awr_ong_tot_raw,w_awr_ong_tot_score,w_awr_ong_tot_cat,w_awr_ong_tot_label,w_awr_ong_tot_weight_fraction,w_awr_smc_qan_raw,w_awr_smc_qan_score,w_awr_smc_qan_cat,w_awr_smc_qan_label,w_awr

In [5]:
kmeans = KMeans(n_clusters=5)

In [12]:
kmeans.fit(data[['bwd_raw']])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [13]:
data['bwd_raw'].describe()

count    62900.000000
mean         0.199848
std          4.936480
min      -1229.884302
25%          0.000967
50%          0.029618
75%          0.291858
max         43.221904
Name: bwd_raw, dtype: float64

In [14]:
data_pos = data.loc[data['bwd_raw'] > 0]

In [15]:
data_pos

,string_id,aq30_id,pfaf_id,gid_1,aqid,gid_0,name_0,name_1,area_km2,bws_raw,bws_score,bws_cat,bws_label,bwd_raw,bwd_score,bwd_cat,bwd_label,iav_raw,iav_score,iav_cat,iav_label,sev_raw,sev_score,sev_cat,sev_label,gtd_raw,gtd_score,gtd_cat,gtd_label,rfr_raw,rfr_score,rfr_cat,rfr_label,cfr_raw,cfr_score,cfr_cat,cfr_label,drr_raw,drr_score,drr_cat,drr_label,ucw_raw,ucw_score,ucw_cat,ucw_label,cep_raw,cep_score,cep_cat,cep_label,udw_raw,udw_score,udw_cat,udw_label,usa_raw,usa_score,usa_cat,usa_label,rri_score,rri_cat,rri_label,w_awr_def_qan_raw,w_awr_def_qan_score,w_awr_def_qan_cat,w_awr_def_qan_label,w_awr_def_qan_weight_fraction,w_awr_def_qal_raw,w_awr_def_qal_score,w_awr_def_qal_cat,w_awr_def_qal_label,w_awr_def_qal_weight_fraction,w_awr_def_rrr_raw,w_awr_def_rrr_score,w_awr_def_rrr_cat,w_awr_def_rrr_label,w_awr_def_rrr_weight_fraction,w_awr_def_tot_raw,w_awr_def_tot_score,w_awr_def_tot_cat,w_awr_def_tot_label,w_awr_def_tot_weight_fraction,w_awr_agr_qan_raw,w_awr_agr_qan_score,w_awr_agr_qan_cat,w_awr_agr_qan_label,w_awr_agr_qan_weight_fraction,w_awr_agr_qal_raw,w_awr_agr_qal_score,w_awr_agr_qal_cat,w_awr_agr_qal_label,w_awr_agr_qal_weight_fraction,w_awr_agr_rrr_raw,w_awr_agr_rrr_score,w_awr_agr_rrr_cat,w_awr_agr_rrr_label,w_awr_agr_rrr_weight_fraction,w_awr_agr_tot_raw,w_awr_agr_tot_score,w_awr_agr_tot_cat,w_awr_agr_tot_label,w_awr_agr_tot_weight_fraction,w_awr_che_qan_raw,w_awr_che_qan_score,w_awr_che_qan_cat,w_awr_che_qan_label,w_awr_che_qan_weight_fraction,w_awr_che_qal_raw,w_awr_che_qal_score,w_awr_che_qal_cat,w_awr_che_qal_label,w_awr_che_qal_weight_fraction,w_awr_che_rrr_raw,w_awr_che_rrr_score,w_awr_che_rrr_cat,w_awr_che_rrr_label,w_awr_che_rrr_weight_fraction,w_awr_che_tot_raw,w_awr_che_tot_score,w_awr_che_tot_cat,w_awr_che_tot_label,w_awr_che_tot_weight_fraction,w_awr_con_qan_raw,w_awr_con_qan_score,w_awr_con_qan_cat,w_awr_con_qan_label,w_awr_con_qan_weight_fraction,w_awr_con_qal_raw,w_awr_con_qal_score,w_awr_con_qal_cat,w_awr_con_qal_label,w_awr_con_qal_weight_fraction,w_awr_con_rrr_raw,w_awr_con_rrr_score,w_awr_con_rrr_cat,w_awr_con_rrr_label,w_awr_con_rrr_weight_fraction,w_awr_con_tot_raw,w_awr_con_tot_score,w_awr_con_tot_cat,w_awr_con_tot_label,w_awr_con_tot_weight_fraction,w_awr_elp_qan_raw,w_awr_elp_qan_score,w_awr_elp_qan_cat,w_awr_elp_qan_label,w_awr_elp_qan_weight_fraction,w_awr_elp_qal_raw,w_awr_elp_qal_score,w_awr_elp_qal_cat,w_awr_elp_qal_label,w_awr_elp_qal_weight_fraction,w_awr_elp_rrr_raw,w_awr_elp_rrr_score,w_awr_elp_rrr_cat,w_awr_elp_rrr_label,w_awr_elp_rrr_weight_fraction,w_awr_elp_tot_raw,w_awr_elp_tot_score,w_awr_elp_tot_cat,w_awr_elp_tot_label,w_awr_elp_tot_weight_fraction,w_awr_fnb_qan_raw,w_awr_fnb_qan_score,w_awr_fnb_qan_cat,w_awr_fnb_qan_label,w_awr_fnb_qan_weight_fraction,w_awr_fnb_qal_raw,w_awr_fnb_qal_score,w_awr_fnb_qal_cat,w_awr_fnb_qal_label,w_awr_fnb_qal_weight_fraction,w_awr_fnb_rrr_raw,w_awr_fnb_rrr_score,w_awr_fnb_rrr_cat,w_awr_fnb_rrr_label,w_awr_fnb_rrr_weight_fraction,w_awr_fnb_tot_raw,w_awr_fnb_tot_score,w_awr_fnb_tot_cat,w_awr_fnb_tot_label,w_awr_fnb_tot_weight_fraction,w_awr_min_qan_raw,w_awr_min_qan_score,w_awr_min_qan_cat,w_awr_min_qan_label,w_awr_min_qan_weight_fraction,w_awr_min_qal_raw,w_awr_min_qal_score,w_awr_min_qal_cat,w_awr_min_qal_label,w_awr_min_qal_weight_fraction,w_awr_min_rrr_raw,w_awr_min_rrr_score,w_awr_min_rrr_cat,w_awr_min_rrr_label,w_awr_min_rrr_weight_fraction,w_awr_min_tot_raw,w_awr_min_tot_score,w_awr_min_tot_cat,w_awr_min_tot_label,w_awr_min_tot_weight_fraction,w_awr_ong_qan_raw,w_awr_ong_qan_score,w_awr_ong_qan_cat,w_awr_ong_qan_label,w_awr_ong_qan_weight_fraction,w_awr_ong_qal_raw,w_awr_ong_qal_score,w_awr_ong_qal_cat,w_awr_ong_qal_label,w_awr_ong_qal_weight_fraction,w_awr_ong_rrr_raw,w_awr_ong_rrr_score,w_awr_ong_rrr_cat,w_awr_ong_rrr_label,w_awr_ong_rrr_weight_fraction,w_awr_ong_tot_raw,w_awr_ong_tot_score,w_awr_ong_tot_cat,w_awr_ong_tot_label,w_awr_ong_tot_weight_fraction,w_awr_smc_qan_raw,w_awr_smc_qan_score,w_awr_smc_qan_cat,w_awr_smc_qan_label,w_awr

In [16]:
kmeans.fit(data_pos[['bwd_raw']])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)